In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv
/kaggle/input/fchls-kernel-matrix/building_block_kernel_v1.npy
/kaggle/input/fchls-kernel-matrix/building_blocks_v1.csv


In [ ]:
!conda install -y -c intel mkl mkl-devel mkl-static mkl-include
!apt-get install -y gfortran
%env MKLROOT=/opt/conda/lib
!pip3 install git+https://github.com/qmlcode/qml@develop --user -U
!pip install duckdb
!pip install molSimplify

Retrieving notices: ...working... done
Channels:
 - intel
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: | 

In [2]:
# import libraries
import duckdb
import pandas as pd
from qml.fchl import get_local_kernels, get_local_symmetric_kernels
from qml import Compound
from sklearn.base import BaseEstimator
from sklearn.kernel_ridge import KernelRidge
from io import StringIO
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import time
import pickle
from molSimplify.Classes.mol3D import mol3D
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

2024-06-11 19:35:39.366314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 19:35:39.366394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 19:35:39.368757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Step 0: FCHL Kernel Building and Storage

##### Note: Skip this step if you already have building_blocks_v1.csv and building_block_kernel_v1.npy available

In [36]:
# extract the 1145 unique building blocks
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

mols = con.query(f"""(SELECT DISTINCT(building_blocks) as building_blocks FROM
((SELECT DISTINCT(buildingblock1_smiles) as building_blocks
                        FROM parquet_scan('{train_path}')
                        )
                        UNION ALL
                        (SELECT DISTINCT(buildingblock2_smiles) as building_blocks
                        FROM parquet_scan('{train_path}'))
                        UNION ALL
                        (SELECT DISTINCT(buildingblock3_smiles) as building_blocks
                        FROM parquet_scan('{train_path}'))))""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
mols.head()

,building_blocks
0,Nc1ccnc(Cl)c1
1,Nc1cncc(F)c1
2,Nc1nc(-c2cccc([N+](=O)[O-])c2)cs1
3,Nc1nc(-c2cccnc2)cs1
4,Nc1nc(Cl)cc(Cl)n1


In [42]:
mols.to_csv('building_blocks_v1.csv', index=True)

In [3]:
# generate the FCHL representations for those building blocks
def make_xyz(mol_smiles):
    mol_3d = mol3D()
    mol_3d.read_smiles(mol_smiles)
    return mol_3d.writexyz('', writestring=True)

def extract_num_atoms(molecule):
    # Split the molecule string by new line and get the first line
    first_line = molecule.split('\n')[0]
    # Convert the first line to an integer (number of atoms)
    return int(first_line)

def make_representation(x, max_atoms):
    # Step 1: Compute the representation
    c = Compound(StringIO(x))
    c.generate_fchl_representation(max_size=max_atoms)
    
    # Step 2: Return it
    return c.representation

class FCHLKernel(BaseEstimator):
    """Class for computing the kernel matrix using the qml utility functions
    
    The input `X` to all of the function is a list of FCHL representation vectors
    
    Follows the "BaseEstimator" API so that we can 
    """
    
    def __init__(self):
        super(FCHLKernel, self).__init__()
        self.train_points = None
    
    def fit(self, X, y=None):
        # Store the training set
        self.train_points = np.array(X)
        return self
        
    def fit_transform(self, X, y=None):
        self.fit(X)
        # Uses the get_localget_local_symmetric_kernels to halve the
        #  computational cost (as the matrix is symmetric)
        return np.squeeze(get_local_symmetric_kernels(self.train_points))
    
    def transform(self, X, y=None):
        return get_local_kernels(np.array(X), self.train_points)[0]

In [7]:
# generate the FCHL representations for those building blocks

# mols['xyz'] = mols['building_blocks'].apply(make_xyz)
# num_atoms = mols['xyz'].apply(extract_num_atoms)
# max_atoms = num_atoms.max()
# mols['fchl_rep'] = mols['xyz'].apply(lambda x: make_representation(x, max_atoms))
# fchl_kernel = FCHLKernel()
# building_block_kernel = fchl_kernel.fit_transform(mols['fchl_rep'].tolist())

In [46]:
np.save('building_block_kernel_v1.npy', building_block_kernel)

Step 1: Sampling training data. We first start simple by randomly sampling N_train molecules from each binding class for a given protein.

In [6]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

train_mols_brd4 = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0 AND protein_name = 'BRD4'
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1 AND protein_name = 'BRD4'
                        ORDER BY random()
                        LIMIT 30000)""").df()

train_mols_hsa = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0 AND protein_name = 'HSA'
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1 AND protein_name = 'HSA'
                        ORDER BY random()
                        LIMIT 30000)""").df()

train_mols_seh = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0 AND protein_name = 'sEH'
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1 AND protein_name = 'sEH'
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
train_mols_brd4.to_csv('sampled_mols_brd4.csv', index=False)
train_mols_hsa.to_csv('sampled_mols_hsa.csv', index=False)
train_mols_seh.to_csv('sampled_mols_seh.csv', index=False)

In [8]:
train_mols_brd4.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,229083705,O=C(Nc1ncc(Br)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,COc1cncc(N)n1,Cl.NCCC(=O)NCc1ccccc1,COc1cncc(Nc2nc(NCCC(=O)NCc3ccccc3)nc(Nc3ncc(Br...,BRD4,0
1,45251562,C[C@@]1(C(=O)O)CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cl.NCC=C(Cl)Cl,Nc1ncnc2c1ncn2C1CCCCO1,C[C@@]1(C(=O)N[Dy])CCCN1c1nc(NCC=C(Cl)Cl)nc(Nc...,BRD4,0
2,166318266,O=C(Nc1cc(C(=O)O)ccc1C(F)(F)F)OCC1c2ccccc2-c2c...,N#Cc1ncc(N)cc1C(F)(F)F,Cn1cc(C(F)F)c(CN)n1,Cn1cc(C(F)F)c(CNc2nc(Nc3cnc(C#N)c(C(F)(F)F)c3)...,BRD4,0
3,20055510,CCC(C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Cc1nnc(CCCN)s1.Cl,Cl.Cl.NCC#Cc1cccnc1,CCC(C)C(Nc1nc(NCC#Cc2cccnc2)nc(NCCCc2nnc(C)s2)...,BRD4,0
4,24375732,CCc1cccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C(=O)O,Cc1ccccc1-n1nc(C)c(CN)c1C.Cl.Cl,CCOC(=O)c1c[nH]nc1N,CCOC(=O)c1c[nH]nc1Nc1nc(NCc2c(C)nn(-c3ccccc3C)...,BRD4,0


Step 2: Construct the FCHL kernel based featurization for each molecule in the training set

In [4]:
bb_kernel = np.load('/kaggle/input/fchls-kernel-matrix/building_block_kernel_v1.npy')

In [5]:
bb_kernel[0,1144]

615.0417279092283

In [6]:
# set up the bb SMILEs -> idx hashmap using the csv file
bb_csv = '/kaggle/input/fchls-kernel-matrix/building_blocks_v1.csv'
bb_df = pd.read_csv(bb_csv)

smiles_to_index = pd.Series(bb_df.index.values, index=bb_df['building_blocks']).to_dict()

In [12]:
# loop through each protein class
# loop through each row in the sampled dataset for that protein class
# get the idx1, idx2 and idx3 for its bb1, bb2 and bb3 building blocks from the hashmap
# now, v1 = [bb_kernel[idx1, j] for j in range(1145)]
# v2 = [bb_kernel[idx2, j] for j in range(1145)]
# v3 = [bb_kernel[idx3, j] for j in range(1145)]
# mol_feature_vector = unit-vector(v1 + v2 + v3) and add this to the sampled dataframe

for sampled_df in [train_mols_brd4, train_mols_hsa, train_mols_seh]:
    mol_features = []
    for _, row in sampled_df.iterrows():
        idx1, idx2, idx3 = smiles_to_index[row['buildingblock1_smiles']], smiles_to_index[row['buildingblock2_smiles']], smiles_to_index[row['buildingblock3_smiles']],
        v1 = np.array([bb_kernel[idx1,i] for i in range(bb_kernel.shape[0])])
        v2 = np.array([bb_kernel[idx2,i] for i in range(bb_kernel.shape[0])])
        v3 = np.array([bb_kernel[idx3,i] for i in range(bb_kernel.shape[0])])
        mol_feature = (v1 + v2 + v3)/np.linalg.norm(v1 + v2 + v3)
        mol_features.append(mol_feature)
    sampled_df['mol_feature'] = mol_features

In [13]:
sampled_df.loc[0, 'mol_feature'].shape

(1145,)

Step 3: Split the sampled molecules into training and validation set

In [14]:
def mols_train_val_split(df, train_frac):
    # Assuming 'df' is your DataFrame
    df_1 = df[df['binds'] == 1]
    df_0 = df[df['binds'] == 0]

    # Randomly sample rows from each group for the training set
    train_1 = df_1.sample(frac=train_frac, random_state=42)
    train_0 = df_0.sample(frac=train_frac, random_state=42)

    # The remaining rows go to the validation set
    val_1 = df_1.drop(train_1.index)
    val_0 = df_0.drop(train_0.index)

    # Concatenate the training and validation sets
    train_set = pd.concat([train_1, train_0])
    val_set = pd.concat([val_1, val_0])

    # Reset the index for a cleaner DataFrame
    train_set = train_set.reset_index(drop=True)
    val_set = val_set.reset_index(drop=True)
    
    return train_set, val_set

In [15]:
brd4_train, brd4_val = mols_train_val_split(train_mols_brd4, train_frac = 0.8)
hsa_train, hsa_val = mols_train_val_split(train_mols_hsa, train_frac = 0.8)
seh_train, seh_val = mols_train_val_split(train_mols_seh, train_frac = 0.8)

In [16]:
brd4_val.loc[0, 'mol_feature'].shape

(1145,)

In [17]:
# loop through each protein class to get its training and validation feature matrix
# where the feature matrix = N_samples, n_features where n_features are taken from the fchl-feature-mol column generated in the previous step
brd4_train_feat_mat = np.stack(brd4_train['mol_feature'].values)
brd4_val_feat_mat = np.stack(brd4_val['mol_feature'].values)
hsa_train_feat_mat = np.stack(hsa_train['mol_feature'].values)
hsa_val_feat_mat = np.stack(hsa_val['mol_feature'].values)
seh_train_feat_mat = np.stack(seh_train['mol_feature'].values)
seh_val_feat_mat = np.stack(seh_val['mol_feature'].values)

brd4_train_target_vec = brd4_train['binds'].values
brd4_val_target_vec = brd4_val['binds'].values
hsa_train_target_vec = hsa_train['binds'].values
hsa_val_target_vec = hsa_val['binds'].values
seh_train_target_vec = seh_train['binds'].values
seh_val_target_vec = seh_val['binds'].values

In [18]:
brd4_train_feat_mat.shape

(48000, 1145)

Step 4: Train 3 separate logistic regression models by optimizing the 'C' (i.e., regularization) hyperparameter

In [19]:
# function for training and returning a model for a given protein class
C_values = np.logspace(-4, 4, 20)

best_C = None
best_val_accuracy = 0
best_model = None

# Iterate over each C value
for C in C_values:
    # Initialize and train the logistic regression model
    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(brd4_train_feat_mat, brd4_train_target_vec)
    
    # Evaluate the model on the validation set
    val_predictions = model.predict(brd4_val_feat_mat)
    val_accuracy = accuracy_score(brd4_val_target_vec, val_predictions)
    
    # Print the current hyperparameter and its validation accuracy
    print(f"C={C}, Validation Accuracy={val_accuracy}")
    
    # Update the best model if current model is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_C = C
        best_model = model

print(f"Best regularization parameter (C): {best_C}")
print(f"Best validation accuracy: {best_val_accuracy}")

C=0.0001, Validation Accuracy=0.5515
C=0.00026366508987303583, Validation Accuracy=0.5520833333333334
C=0.0006951927961775605, Validation Accuracy=0.55125
C=0.0018329807108324356, Validation Accuracy=0.5505833333333333
C=0.004832930238571752, Validation Accuracy=0.5504166666666667
C=0.012742749857031334, Validation Accuracy=0.5501666666666667
C=0.03359818286283781, Validation Accuracy=0.55025
C=0.08858667904100823, Validation Accuracy=0.55025
C=0.23357214690901212, Validation Accuracy=0.55025
C=0.615848211066026, Validation Accuracy=0.55025
C=1.623776739188721, Validation Accuracy=0.55025
C=4.281332398719396, Validation Accuracy=0.55025
C=11.288378916846883, Validation Accuracy=0.55025
C=29.763514416313132, Validation Accuracy=0.55025
C=78.47599703514607, Validation Accuracy=0.55025
C=206.913808111479, Validation Accuracy=0.5503333333333333
C=545.5594781168514, Validation Accuracy=0.5589166666666666
C=1438.44988828766, Validation Accuracy=0.55025
C=3792.690190732246, Validation Accurac

In [20]:
# function for training and returning a model for a given protein class
C_values = np.logspace(-4, 4, 20)

best_C = None
best_val_accuracy = 0
best_model = None

# Iterate over each C value
for C in C_values:
    # Initialize and train the logistic regression model
    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(hsa_train_feat_mat, hsa_train_target_vec)
    
    # Evaluate the model on the validation set
    val_predictions = model.predict(hsa_val_feat_mat)
    val_accuracy = accuracy_score(hsa_val_target_vec, val_predictions)
    
    # Print the current hyperparameter and its validation accuracy
    print(f"C={C}, Validation Accuracy={val_accuracy}")
    
    # Update the best model if current model is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_C = C
        best_model = model

print(f"Best regularization parameter (C): {best_C}")
print(f"Best validation accuracy: {best_val_accuracy}")

C=0.0001, Validation Accuracy=0.582
C=0.00026366508987303583, Validation Accuracy=0.581
C=0.0006951927961775605, Validation Accuracy=0.5806666666666667
C=0.0018329807108324356, Validation Accuracy=0.5805
C=0.004832930238571752, Validation Accuracy=0.5801666666666667
C=0.012742749857031334, Validation Accuracy=0.5801666666666667
C=0.03359818286283781, Validation Accuracy=0.5804166666666667
C=0.08858667904100823, Validation Accuracy=0.5801666666666667
C=0.23357214690901212, Validation Accuracy=0.5804166666666667
C=0.615848211066026, Validation Accuracy=0.5804166666666667
C=1.623776739188721, Validation Accuracy=0.5804166666666667
C=4.281332398719396, Validation Accuracy=0.5805833333333333
C=11.288378916846883, Validation Accuracy=0.5805
C=29.763514416313132, Validation Accuracy=0.5805833333333333
C=78.47599703514607, Validation Accuracy=0.5804166666666667
C=206.913808111479, Validation Accuracy=0.5845
C=545.5594781168514, Validation Accuracy=0.58925
C=1438.44988828766, Validation Accurac

In [21]:
# function for training and returning a model for a given protein class
C_values = np.logspace(-4, 4, 20)

best_C = None
best_val_accuracy = 0
best_model = None

# Iterate over each C value
for C in C_values:
    # Initialize and train the logistic regression model
    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(seh_train_feat_mat, seh_train_target_vec)
    
    # Evaluate the model on the validation set
    val_predictions = model.predict(seh_val_feat_mat)
    val_accuracy = accuracy_score(seh_val_target_vec, val_predictions)
    
    # Print the current hyperparameter and its validation accuracy
    print(f"C={C}, Validation Accuracy={val_accuracy}")
    
    # Update the best model if current model is better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_C = C
        best_model = model

print(f"Best regularization parameter (C): {best_C}")
print(f"Best validation accuracy: {best_val_accuracy}")

C=0.0001, Validation Accuracy=0.6673333333333333
C=0.00026366508987303583, Validation Accuracy=0.66725
C=0.0006951927961775605, Validation Accuracy=0.66675
C=0.0018329807108324356, Validation Accuracy=0.6668333333333333
C=0.004832930238571752, Validation Accuracy=0.6669166666666667
C=0.012742749857031334, Validation Accuracy=0.6669166666666667
C=0.03359818286283781, Validation Accuracy=0.6669166666666667
C=0.08858667904100823, Validation Accuracy=0.6669166666666667
C=0.23357214690901212, Validation Accuracy=0.6669166666666667
C=0.615848211066026, Validation Accuracy=0.6669166666666667
C=1.623776739188721, Validation Accuracy=0.6669166666666667
C=4.281332398719396, Validation Accuracy=0.6669166666666667
C=11.288378916846883, Validation Accuracy=0.6669166666666667
C=29.763514416313132, Validation Accuracy=0.6671666666666667
C=78.47599703514607, Validation Accuracy=0.6675
C=206.913808111479, Validation Accuracy=0.6686666666666666
C=545.5594781168514, Validation Accuracy=0.6696666666666666

In [22]:
# collect all trained models for each protein class in a list
brd4_model = LogisticRegression(C=545.5594781168514, max_iter=10000)
brd4_combined_feat_mat = np.vstack((brd4_train_feat_mat, brd4_val_feat_mat))
brd4_combined_target_vec = np.concatenate((brd4_train_target_vec, brd4_val_target_vec))
brd4_model.fit(brd4_combined_feat_mat, brd4_combined_target_vec)

hsa_model = LogisticRegression(C=10000.0, max_iter=10000)
hsa_combined_feat_mat = np.vstack((hsa_train_feat_mat, hsa_val_feat_mat))
hsa_combined_target_vec = np.concatenate((hsa_train_target_vec, hsa_val_target_vec))
hsa_model.fit(hsa_combined_feat_mat, hsa_combined_target_vec)

seh_model = LogisticRegression(C=10000.0, max_iter=10000)
seh_combined_feat_mat = np.vstack((seh_train_feat_mat, seh_val_feat_mat))
seh_combined_target_vec = np.concatenate((seh_train_target_vec, seh_val_target_vec))
seh_model.fit(seh_combined_feat_mat, seh_combined_target_vec)


LogisticRegression(C=10000.0, max_iter=10000)

In [24]:
# Save the model
with open('brd4_logistic.pkl', 'wb') as file:
    pickle.dump(brd4_model, file)

In [25]:
# Save the model
with open('hsa_logistic.pkl', 'wb') as file:
    pickle.dump(hsa_model, file)

In [26]:
# Save the model
with open('seh_logistic.pkl', 'wb') as file:
    pickle.dump(seh_model, file)

In [11]:
del train_mols_brd4
del train_mols_hsa
del train_mols_seh
del sampled_df
del brd4_train, brd4_val
del hsa_train, hsa_val
del seh_train, seh_val
del brd4_train_feat_mat
del brd4_val_feat_mat
del hsa_train_feat_mat
del hsa_val_feat_mat
del seh_train_feat_mat
del seh_val_feat_mat
del brd4_train_target_vec
del brd4_val_target_vec
del hsa_train_target_vec
del hsa_val_target_vec
del seh_train_target_vec
del seh_val_target_vec

NameError: name 'train_mols_brd4' is not defined

Step 5: Test Set Results Generate

In [7]:
# divide the test set into the respective protein classes
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

test_mols_brd4 = con.query(f"""(SELECT *
                        FROM parquet_scan('{test_path}')
                        WHERE protein_name = 'BRD4'
                        LIMIT 1
                        )""").df()

# test_mols_hsa = con.query(f"""(SELECT *
#                         FROM parquet_scan('{test_path}')
#                         WHERE protein_name = 'HSA'
#                         )""").df()

# test_mols_seh = con.query(f"""(SELECT *
#                         FROM parquet_scan('{test_path}')
#                         WHERE protein_name = 'sEH'
#                         )""").df()

con.close()

In [8]:
test_mols_brd4.shape

(1, 6)

In [9]:
test_mols_brd4

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4


In [10]:
# test_mols_hsa.shape

In [11]:
# test_mols_seh.shape

In [12]:
bb_df.head()

,Unnamed: 0,building_blocks
0,0,Nc1ccnc(Cl)c1
1,1,Nc1cncc(F)c1
2,2,Nc1nc(-c2cccc([N+](=O)[O-])c2)cs1
3,3,Nc1nc(-c2cccnc2)cs1
4,4,Nc1nc(Cl)cc(Cl)n1


In [20]:
bb_df['xyz'] = bb_df['building_blocks'].apply(make_xyz)
num_atoms = bb_df['xyz'].apply(extract_num_atoms)
max_atoms = num_atoms.max()
bb_df['fchl_rep'] = bb_df['xyz'].apply(lambda x: make_representation(x, max_atoms))
# fchl_kernel = FCHLKernel()
# fchl_kernel.fit(bb_df['fchl_rep'].tolist())
bb_fchls = bb_df['fchl_rep'].tolist()

FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup

In [25]:
type(bb_fchls)

list

In [26]:
with open('bb_fchl_list.pkl', 'wb') as file:
    pickle.dump(bb_fchls, file)

In [13]:
with open('/kaggle/working/bb_fchl_list.pkl', 'rb') as file:
    bb_fchls = pickle.load(file)

In [16]:
mol_bb1 = test_mols_brd4.loc[0, 'buildingblock1_smiles']
mol_bb2 = test_mols_brd4.loc[0, 'buildingblock2_smiles']
mol_bb3 = test_mols_brd4.loc[0, 'buildingblock3_smiles']

In [17]:
test_bb_hashmap = {}

In [29]:
bb1_xyz = make_xyz(mol_bb1)
bb1_fchl = make_representation(bb1_xyz, max_atoms)
get_local_kernels(np.array([bb1_fchl]),np.array(bb_fchls))[0]

array([[[ 575.9157754 ,  575.98104006,  975.85421886, ...,
         2224.00794179, 2575.73137267, 2271.14037441]]])

In [ ]:
bb2_xyz = make_xyz(mol_bb2)
bb2_fchl = make_representation(bb2_xyz, max_atoms)
get_local_kernels(np.array([bb2_fchl]),np.array(bb_fchls))

In [1]:
v2 = fchl_kernel.transform([bb2_fchl])

NameError: name 'fchl_kernel' is not defined

In [ ]:
bb2_xyz = make_xyz(mol_bb2)
print("Line 1 done")
bb2_fchl = make_representation(bb2_xyz, max_atoms)
print("Line 2 done")
v2 = fchl_kernel.transform([bb2_fchl])
print("Line 3 done")
v2 = np.squeeze(v2.T)
print("Line 4 done")
test_bb_hashmap[mol_bb2] = v2
print("Line 5 done")

Line 1 done
Line 2 done


In [ ]:
# for a given protein class
# loop through each row
# check if bb1, bb2 and bb3 belong to the bb-mols-set
# for those that do, get their FCHL vectors from the bb-kernel using their idx's from the hashmap 
# for those that do not, get their FCHL vectors from scratch using the transform function
# add this new bb to a new test-bb-hashmap with its feature vector already stored
# get the mol feature vector

test_bb_hashmap = {}

for test_df in [test_mols_brd4]:
    mol_features = []
    for _, row in test_df.iterrows():
        mol_bb1 = row['buildingblock1_smiles']
        mol_bb2 = row['buildingblock2_smiles']
        mol_bb3 = row['buildingblock3_smiles']
        if mol_bb1 in smiles_to_index:
            idx1 = smiles_to_index[mol_bb1]
            v1 = np.array([bb_kernel[idx1,i] for i in range(bb_kernel.shape[0])])
        elif mol_bb1 in test_bb_hashmap:
            v1 = test_bb_hashmap[mol_bb1]
        else:
            bb1_xyz = make_xyz(mol_bb1)
            time.sleep(0.5)
            bb1_fchl = make_representation(bb1_xyz, max_atoms)
            time.sleep(0.5)
            v1 = fchl_kernel.transform([bb1_fchl])
            time.sleep(0.5)
            v1 = np.squeeze(v1.T)
            test_bb_hashmap[mol_bb1] = v1
        if mol_bb2 in smiles_to_index:
            idx2 = smiles_to_index[mol_bb2]
            v2 = np.array([bb_kernel[idx2,i] for i in range(bb_kernel.shape[0])])
        elif mol_bb2 in test_bb_hashmap:
            v2 = test_bb_hashmap[mol_bb2]
        else:
            bb2_xyz = make_xyz(mol_bb2)
            time.sleep(0.5)
            bb2_fchl = make_representation(bb2_xyz, max_atoms)
            time.sleep(0.5)
            v2 = fchl_kernel.transform([bb2_fchl])
            time.sleep(0.5)
            v2 = np.squeeze(v2.T)
            test_bb_hashmap[mol_bb2] = v2
        if mol_bb3 in smiles_to_index:
            idx3 = smiles_to_index[mol_bb3]
            v3 = np.array([bb_kernel[idx3,i] for i in range(bb_kernel.shape[0])])
        elif mol_bb3 in test_bb_hashmap:
            v3 = test_bb_hashmap[mol_bb3]
        else:
            bb3_xyz = make_xyz(mol_bb3)
            time.sleep(0.5)
            bb3_fchl = make_representation(bb3_xyz, max_atoms)
            time.sleep(0.5)
            v3 = fchl_kernel.transform([bb3_fchl])
            time.sleep(0.5)
            v3 = np.squeeze(v3.T)
            test_bb_hashmap[mol_bb3] = v3
        mol_feature = (v1 + v2 + v3)/np.linalg.norm(v1 + v2 + v3)
        mol_features.append(mol_feature)
    test_df['mol_feature'] = mol_features

In [1]:
max_atoms

NameError: name 'max_atoms' is not defined

In [ ]:
# generate the feature matrix for the test set
# feed to the model
# generate probability scores and store it at the corresponding id in a new column in the full test set dataframe
# generate csv from the full test set dataframe, only for the id and for the probability scores column
# submit!